In [ ]:
import skimage
import os
import pydicom
from matplotlib import pyplot as plt
import skimage.feature
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

dirname = "../SagittalClassificationLocal/Temp" #npy files dir name

In [ ]:
filenames = [el for el in os.listdir(dirname) if el.endswith('.npy')]
dataset = np.zeros((len(filenames), 486), dtype=np.float32)
for i, filename in enumerate(filenames):
    img = np.load(os.path.join(dirname, filename))[:,:, 0]
    smooth_img = skimage.filters.gaussian(img, 10)
    sobel_img = skimage.filters.sobel(smooth_img)
    if sobel_img.shape[0] == 432:
        hog, hog_image = skimage.feature.hog(skimage.transform.rescale(sobel_img[:,:], 1/11), orientations=9, visualize=True)
    else:
        hog, hog_image = skimage.feature.hog(skimage.transform.rescale(sobel_img[:,:], 1/10), orientations=9, visualize=True)
    dataset[i] = hog

# PCA

In [ ]:
pca = PCA(n_components=8)
pca.fit(dataset)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
pca_dataset = pca.transform(dataset)

# KMeans Clustering

In [ ]:
kmeans = KMeans(n_clusters=2)
kmeans.fit(pca_dataset)

In [ ]:
kmeans.labels_

In [ ]:
for i, filename in enumerate(filenames):
    print("Label: ", kmeans.labels_[i])
    img = np.load(dirname + "/" + filename)
    plt.imshow(img)
    plt.show()